In [1]:
# Re-import pandas since the environment reset
import pandas as pd

# Load the updated CSV file
file_path = "14.csv"
df = pd.read_csv(file_path)

# Display the first few rows to confirm the structure
df.head()

,srcStateName,Crop name,srcDistrictName,Crop season,Land area utilized for production,Crop production,Crop yield
0,GUJARAT,Sesamum,PATAN,Summer,3.0,NaN,0.00
1,PUNJAB,Maize,SANGRUR,Kharif,1000.0,4000.0,4.00
2,PUNJAB,Maize,TARN TARAN,Kharif,1000.0,4000.0,4.00
3,PUNJAB,Masoor,GURDASPUR,Rabi,200.0,140.0,0.70
4,PUNJAB,Masoor,HOSHIARPUR,Rabi,200.0,150.0,0.75


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import numpy as np

In [3]:
df = df.dropna(subset=["Crop yield"])

# Define target and features
X = df.drop(columns=["Crop yield"])
y = df["Crop yield"]

# Identify column types
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numerical_cols = X.select_dtypes(include=["float64", "int64"]).columns.tolist()


In [4]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_cols),
        ("cat", categorical_transformer, categorical_cols)
    ]
)

In [5]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0)
}

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
results = {}
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    results[name] = {"RMSE": rmse, "R2": r2, "model": pipeline}

results

{'Linear Regression': {'RMSE': 427.33121681541894,
  'R2': 0.8119494863971893,
  'model': Pipeline(steps=[('preprocessor',
                   ColumnTransformer(transformers=[('num',
                                                    Pipeline(steps=[('imputer',
                                                                     SimpleImputer()),
                                                                    ('scaler',
                                                                     StandardScaler())]),
                                                    ['Land area utilized for '
                                                     'production',
                                                     'Crop production']),
                                                   ('cat',
                                                    Pipeline(steps=[('imputer',
                                                                     SimpleImputer(strategy='most_frequent')),
            

In [7]:
import joblib

best_rf_model = results['Random Forest']['model']

joblib.dump(best_rf_model, 'best_rf_yield.pkl')

['best_rf_yield.pkl']